In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator
import os

def plot_seaborn(score_dict, ours_test_score, test_on, metric, solver_name, solver_label, marker=['o', 's', 'D']):
    # Your data
    plot_step = [10, 25, 50, 75, 100]
    plt.figure(figsize=(14, 12))
    plt.rcParams['font.family'] = 'Calibri'
    sns.set_theme(style="whitegrid")  # Set the theme for Seaborn
    sns.set_context("talk")  # Adjusts the size of the labels, lines, and other elements of the plot# Adjusting text elements
    
    linewidth = 4
    markersize = 13
    fontsize = 30
    tick_fontsize = 30
    

    for i, sn in enumerate(solver_name):
        x = []
        y = []
        for steps in plot_step:
            error = score_dict[sn]['steps'][str(steps)]
            x.append(int(steps))
            y.append(float(error[f'mean_{metric}']))

        sns.lineplot(x=x, y=y, marker=marker[i], linestyle='-', label=solver_label[i], linewidth=linewidth, markersize=markersize)

    # Add titles and labels
    # plt.title(f'{str.upper(metric)} vs. Diffusion steps', fontsize=35)
    plt.xlabel('Diffusion steps (Runtime in sec)', fontsize=fontsize, labelpad=10)
    plt.ylabel(f'{str.upper(metric)}↓', fontsize=fontsize, labelpad=10)
    plt.xticks(fontsize=tick_fontsize)  # Adjust X-axis tick label size
    plt.yticks(fontsize=tick_fontsize)  # Adjust Y-axis tick label size

    plt.tick_params(axis='x', which='major', pad=-5)  # Increase pad for x-axis
    plt.tick_params(axis='y', which='major', pad=-5)  # Increase pad for y-axis
    if metric == 'dssim':
        current_yticks_values, current_yticks_labels = plt.yticks()
        plt.ylim(bottom=current_yticks_values[0] - 0.001)
    
    # current_yticks_values = list(np.array(current_yticks_values) + 0.001)  # Adjust Y-axis tick values
    # plt.yticks(current_yticks_values, [f"{y:.4f}" for y in current_yticks_values])  # Adjust Y-axis tick labels


    runtime_list = [f"{time['mean_runtime']:.2f}" for step, time in runtime_dict.items()]
    print(runtime_list)
    plt.xticks(x, [f"{x_val}\n(~{rt})" for x_val, rt in zip(x, runtime_list)])  # Adjust runtime text here
    plt.gca().yaxis.set_major_locator(MaxNLocator(nbins=5))  # Adjust `nbins` for y-axis ticks
    

    # Adding DiFaReli's baseline
    plt.axhline(y=ours_test_score['difareli++'][f'mean_{metric}'], color='magenta', linestyle='--', label='DiFaReli (1000 steps)', linewidth=3, alpha=0.9)
    if metric == 'lpips':
        plt.text(80, ours_test_score['difareli++'][f'mean_{metric}'], '~194.26 sec', 
            ha='left', va='bottom', color='magenta', fontsize=28,
            bbox=dict(facecolor='white', alpha=0.4, edgecolor=None, boxstyle='round,pad=0.2'))
    else: 
        plt.text(6.5, ours_test_score['difareli++'][f'mean_{metric}'], '~194.26 sec', 
            ha='left', va='bottom', color='magenta', fontsize=28,
            bbox=dict(facecolor='white', alpha=0.4, edgecolor=None, boxstyle='round,pad=0.2'))
        
    # plt.text(6, ours_test_score['difareli'][f'mean_{metric}'], '~194.26 sec', ha='left', va='bottom', color='magenta', fontsize=28)
    # plt.text(80, ours_test_score['difareli'][f'mean_{metric}'], '~194.26 sec', ha='left', va='bottom', color='magenta', fontsize=28)

    # Adding single shot (assuming `ours_test_score` is defined outside this function)
    plt.axhline(y=ours_test_score['ss'][f'mean_{metric}'], color='red', linestyle='--', label='DiFaReli$_{\\mathbf{ss}}$ (1 step)', linewidth=linewidth, alpha=0.9)
    # plt.text(6.5, ours_test_score['ss'][f'mean_{metric}'], '~0.09 sec', ha='left', va='bottom', color='red', fontsize=tick_fontsize)
    
    if metric == 'lpips':
        plt.text(80, ours_test_score['ss'][f'mean_{metric}'], '~0.09 sec', 
            ha='left', va='bottom', color='red', fontsize=28,
            bbox=dict(facecolor='white', alpha=0.4, edgecolor=None, boxstyle='round,pad=0.2'))
    else: 
        plt.text(6.5, ours_test_score['ss'][f'mean_{metric}'], '~0.09 sec', 
            ha='left', va='bottom', color='red', fontsize=28,
            bbox=dict(facecolor='white', alpha=0.4, edgecolor=None, boxstyle='round,pad=0.2'))
    # plt.text(80, ours_test_score['ss'][f'mean_{metric}'], '~0.09 sec', ha='left', va='bottom', color='red', fontsize=tick_fontsize)

    leg = plt.legend(fontsize=34)  # Adjust legend font size
    for text in leg.get_texts():
        if text.get_text() == 'DiFaReli$_{\\mathbf{ss}}$ (1 step)':
            # text.set_color('red')
            text.set_fontweight('bold')
    
    os.makedirs('./roc_curve', exist_ok=True)
    plt.savefig(f'./roc_curve/{test_on}_{metric}.png', bbox_inches='tight', dpi=300)
    plt.show()
    

In [27]:
import numpy as np
import json, glob, os
import matplotlib.pyplot as plt

test_on = 'mp_test'
depth_scale = 100.0
path = f'/data/mint/TPAMI_evaluations/MP/score/ours_difareli++_with_solver/multipie/{test_on}/'
runtime_path = f'/data/mint/TPAMI_evaluations/MP/pred/Ours/ours_difareli++_with_solver/ema_085000/{test_on}/'
ours_runtime_path = f'/data/mint/TPAMI_evaluations/MP/pred/Ours/ours_difareli++_single_shot_eval/ema_300000/{test_on}_data2/'   # Cause we run this on v20 data2

score_dict = {'dpmsolver++': {'steps':{}}, 'dpmsolver': {'steps':{}}, 'unipc': {'steps':{}}}
for p in glob.glob(path + '*.json'):
    if str(depth_scale) in p:
        solver_cfg = p.split('/')[-1].split('.json')[0]
        _, _, solver_name, solver_method, solver_step, solver_order, _, solver_depth_scale = solver_cfg.split('_')
        # score_dict[solver_name]['steps'][int(solver_step)] = {}
        score_dict[solver_name]['steps'][solver_step] = {}
        # print(solver_name, solver_method, solver_step, solver_order, solver_depth_scale)
        with open(p, 'r') as f:
            data = json.load(f)
            for metric in ['dssim', 'mse', 'lpips']:
                # print(data['eval_score'][score].split('+-'))
                mean, sd = data['eval_score'][metric].split('+-')
                mean = float(mean)
                sd = float(sd)
                score_dict[solver_name]['steps'][solver_step][f'mean_{metric}'] = mean
                score_dict[solver_name]['steps'][solver_step][f'sd_{metric}'] = sd
            runtime_file = glob.glob(f"{runtime_path}/{solver_cfg.split('eval_score_')[-1]}/runtime*")
            # print(runtime_file, f'{runtime_path}/{solver_cfg}')
            with open(runtime_file[0], 'r') as f:
                runtime = json.load(f)
            score_dict[solver_name]['steps'][str(solver_step)]['mean_runtime'] = np.mean(np.array(runtime['rev_time']) + np.array(runtime['relit_time']))
            score_dict[solver_name]['steps'][str(solver_step)]['sd_runtime'] = np.std(np.array(runtime['rev_time']) + np.array(runtime['relit_time']))
                

runtime_dict = {}
plot_step = ['10', '25', '50', '75', '100']
for step in plot_step:
    tmp = []
    for solver in score_dict.keys():
        tmp.append(score_dict[solver]['steps'][str(step)]['mean_runtime'])
        # print(solver, step, score_dict[solver]['steps'][str(step)]['mean_runtime'])
    runtime_dict[step] = {'mean_runtime':np.mean(tmp), 'sd_runtime':(np.mean(tmp))}
 
 
# Single-shot runtime   
ours_runtime_file = glob.glob(f"{ours_runtime_path}/SD{depth_scale}/runtime*")
assert len(ours_runtime_file) == 1
ours_runtime_dict = {}
with open(ours_runtime_file[0], 'r') as f:
    ours_runtime = json.load(f)
# print(ours_runtime.keys())
ours_runtime_dict['ss'] = ours_runtime['mean_relit_time']

# DDIM 1000 steps runtime
ours_runtime_file = glob.glob(f"{ours_runtime_path}/SD{depth_scale}/runtime*")
assert len(ours_runtime_file) == 1
ours_runtime_dict = {}
with open(ours_runtime_file[0], 'r') as f:
    ours_runtime = json.load(f)
# print(ours_runtime.keys())
ours_runtime_dict['ss'] = ours_runtime['mean_relit_time']


assert False

ours_single_shot = {
    'mean_mse': 0.0075,
    'sd_mse': 0.0077,
    'mean_lpips': 0.1023,
    'sd_lpips': 0.0367,
    'mean_dssim': 0.0590,
    'sd_dssim': 0.0272,
}

ours_ddim_1000ts = {
    'mean_mse': 0.0090,
    'sd_mse': 0.0095,
    'mean_lpips': 0.1043,
    'sd_lpips': 0.0396,
    'mean_dssim': 0.0604,
    'sd_dssim': 0.0296,
}

ours_test_score = {'difareli++':ours_ddim_1000ts, 'ss':ours_single_shot}

plot_seaborn(score_dict, ours_test_score, test_on, 'mse', ['dpmsolver', 'dpmsolver++', 'unipc'], ['DPM-Solver', 'DPM-Solver++', 'UniPC'])
plot_seaborn(score_dict, ours_test_score, test_on, 'dssim', ['dpmsolver', 'dpmsolver++', 'unipc'], ['DPM-Solver', 'DPM-Solver++', 'UniPC'])
plot_seaborn(score_dict, ours_test_score, test_on, 'lpips', ['dpmsolver', 'dpmsolver++', 'unipc'], ['DPM-Solver', 'DPM-Solver++', 'UniPC'])

dict_keys(['relit_time', 'sub_relit_time', 'render_time', 'name', 'mean_relit_time', 'std_relit_time', 'all_relit_time', 'all_render_time', 'set', 'n_sj'])
0.05445445081750644
[11.969620943069458, 7.440147876739502, 6.856574058532715, 7.041562080383301, 6.637836933135986, 6.970042943954468, 6.642632961273193, 7.040046453475952, 6.909297704696655, 6.926597595214844, 6.786064386367798, 6.954662322998047, 7.01051664352417, 7.017429351806641, 7.0364179611206055, 6.737760305404663, 7.393414735794067, 6.812155485153198, 7.076831340789795, 6.879961252212524, 6.752802848815918, 6.832024812698364, 7.013914108276367, 7.119405031204224, 7.119612216949463, 7.0092737674713135, 7.04060173034668, 6.734063386917114, 6.960580110549927, 6.876909255981445, 6.893086194992065, 6.878818511962891, 7.17295241355896, 6.889362812042236, 6.915526390075684, 6.70533561706543, 7.0844221115112305, 6.771041393280029, 7.310041189193726, 7.178426027297974, 6.852252006530762, 7.120975494384766, 7.265005350112915, 7.1197

AssertionError: 

In [ ]:
import numpy as np
import json, glob, os
import matplotlib.pyplot as plt

test_on = 'mp_test2'
path = f'/data/mint/TPAMI_evaluations/MP/score/ours_difareli++_with_solver/multipie/{test_on}/'
runtime_path = f'/data/mint/TPAMI_evaluations/MP/pred/Ours/ours_difareli++_with_solver/ema_085000/{test_on}/'
depth_scale = 100.0

score_dict = {'dpmsolver++': {'steps':{}}, 'dpmsolver': {'steps':{}}, 'unipc': {'steps':{}}}
for p in glob.glob(path + '*.json'):
    if str(depth_scale) in p:
        solver_cfg = p.split('/')[-1].split('.json')[0]
        _, _, solver_name, solver_method, solver_step, solver_order, _, solver_depth_scale = solver_cfg.split('_')
        # score_dict[solver_name]['steps'][int(solver_step)] = {}
        score_dict[solver_name]['steps'][solver_step] = {}
        # print(solver_name, solver_method, solver_step, solver_order, solver_depth_scale)
        with open(p, 'r') as f:
            data = json.load(f)
            for metric in ['dssim', 'mse', 'lpips']:
                # print(data['eval_score'][score].split('+-'))
                mean, sd = data['eval_score'][metric].split('+-')
                mean = float(mean)
                sd = float(sd)
                score_dict[solver_name]['steps'][solver_step][f'mean_{metric}'] = mean
                score_dict[solver_name]['steps'][solver_step][f'sd_{metric}'] = sd
            runtime_file = glob.glob(f"{runtime_path}/{solver_cfg.split('eval_score_')[-1]}/runtime*")
            # print(runtime_file, f'{runtime_path}/{solver_cfg}')
            with open(runtime_file[0], 'r') as f:
                runtime = json.load(f)
            score_dict[solver_name]['steps'][str(solver_step)]['mean_runtime'] = np.mean(np.array(runtime['rev_time']) + np.array(runtime['relit_time']))
            score_dict[solver_name]['steps'][str(solver_step)]['sd_runtime'] = np.std(np.array(runtime['rev_time']) + np.array(runtime['relit_time']))
                

runtime_dict = {}
plot_step = ['10', '25', '50', '75', '100']
for step in plot_step:
    tmp = []
    for solver in score_dict.keys():
        tmp.append(score_dict[solver]['steps'][str(step)]['mean_runtime'])
        # print(solver, step, score_dict[solver]['steps'][str(step)]['mean_runtime'])
    runtime_dict[step] = {'mean_runtime':np.mean(tmp), 'sd_runtime':(np.mean(tmp))}
# print(runtime_dict)

ours_single_shot = {
    'mean_mse': 0.0148,
    'sd_mse': 0.0149,
    'mean_lpips': 0.1123,
    'sd_lpips': 0.0477,
    'mean_dssim': 0.0801,
    'sd_dssim': 0.0418,
}

ours_ddim_1000ts = {
    'mean_mse': 0.0169,
    'sd_mse': 0.0173,
    'mean_lpips': 0.1223,
    'sd_lpips': 0.0506,
    'mean_dssim': 0.0824,
    'sd_dssim': 0.0452,
}

ours_test_score = {'difareli++':ours_ddim_1000ts, 'ss':ours_single_shot}

plot_seaborn(score_dict, ours_test_score, test_on, 'mse', ['dpmsolver', 'dpmsolver++', 'unipc'], ['DPM-Solver', 'DPM-Solver++', 'UniPC'])
plot_seaborn(score_dict, ours_test_score, test_on, 'dssim', ['dpmsolver', 'dpmsolver++', 'unipc'], ['DPM-Solver', 'DPM-Solver++', 'UniPC'])
plot_seaborn(score_dict, ours_test_score, test_on, 'lpips', ['dpmsolver', 'dpmsolver++', 'unipc'], ['DPM-Solver', 'DPM-Solver++', 'UniPC'])

In [37]:
# Examine example of runtime for single-shot
example_runtime = '/data/mint/sampling/TPAMI/main_result/ffhq/moving_shadows/top_100_min_shadows/log=paired+difareli+cs+nodpm+trainset_256_cfg=paired+difareli+cs+nodpm+trainset_256.yaml_rot2_randmaxC/runtime_20241011-160420.json'
with open(example_runtime, 'r') as f:
    ex_rt = json.load(f)
print(ex_rt.keys())
# sub_pure_render_deca = np.stack(ex_rt['sub_pure_render_deca_time'])
# sub_pure_render_shadow_time = np.stack(ex_rt['sub_pure_render_shadow_time'])
sub_pure_render_deca = ex_rt['all_pure_render_deca_time']
sub_pure_render_shadow_time = ex_rt['all_pure_render_shadow_time']

print(sub_pure_render_deca, sub_pure_render_shadow_time)
print(sub_pure_render_deca.shape, sub_pure_render_shadow_time.shape)
n_frames = 60
print("[#] Mean = {}")
    

dict_keys(['relit_time', 'sub_relit_time', 'render_time', 'sub_pure_render_deca_time', 'sub_pure_render_shadow_time', 'sub_load_deca_for_shadow_time', 'sub_load_deca_time', 'name', 'mean_relit_time', 'std_relit_time', 'all_relit_time', 'all_render_time', 'all_pure_render_deca_time', 'all_pure_render_shadow_time', 'all_load_deca_time', 'all_load_deca_for_shadow_time', 'set', 'n_sj'])


AttributeError: 'NoneType' object has no attribute 'shape'